### Preparar ambiente

In [1]:
import enum
print(enum.__file__)  
# standard library location should be something like 
# /usr/local/lib/python3.6/enum.py

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/enum.py


In [2]:
import sys
!{sys.executable} -m pip uninstall -y enum34

In [3]:
!{sys.executable} -m pip install awswrangler

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [4]:
!{sys.executable} -m pip install PyAthena

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


### Conectar con pyathena

In [5]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

In [6]:
import pandas as pd
import awswrangler as wr
import numpy as np
import calendar
import boto3
s3 = boto3.resource('s3')

In [7]:
import os
import zipfile
import shutil
from io import StringIO
import io

### Declaración funciones para Impo-Expo s3

In [8]:
def write_dataframe_to_csv_on_s3(dataframe, bucket, filename):
    """ Write a dataframe to a CSV on S3 """
    # Create buffer
    csv_buffer = StringIO()
    # Write dataframe to buffer
    dataframe.to_csv(csv_buffer, sep=";" , line_terminator='\n')
    # Create S3 object
    s3_resource = boto3.resource("s3")
    # Write buffer to S3 object
    s3_resource.Object(bucket, filename).put(Body=csv_buffer.getvalue())
    print("Writing {} records to {}".format(len(dataframe), filename))
    print("S3")
    print(dataframe.shape)
    

In [9]:
def write_dataframe_to_excel_on_s3(dataframe, bucket, filename):
    """ Write a dataframe to a excel on S3 """
    with io.BytesIO() as output:
        with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
            dataframe.to_excel(writer,index=False)
        data = output.getvalue()

    s3 = boto3.resource('s3')
    s3.Bucket(bucket).put_object(Key=filename, Body=data)
    

In [10]:
def import_dataframe_csv_from_s3(bucket, filename):
    client = boto3.client('s3')
    bucket_name = bucket
    object_key = filename
    csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
    body = csv_obj['Body']
    csv_string = body.read().decode('utf-8')
    df = pd.read_csv(StringIO(csv_string),sep=";")
    return df

In [11]:
def import_dataframe_excel_from_s3(bucket, filename):
    client = boto3.client('s3')
    bucket_name = bucket
    object_key = filename
    obj = client.get_object(Bucket=bucket_name, Key=object_key)
    data = obj['Body'].read()
    df = pd.read_excel(io.BytesIO(data), encoding='utf-8')
    return df

In [12]:
def write_file_to_s3(bucket,file_local,file_s3):
    s3.Bucket(bucket).upload_file(file_local,file_s3)

### Crea tabla de indicadores por dia

In [13]:
fecha_inicio = '09-01-2020'
fecha_final = '09-30-2020'
fechas = pd.date_range(fecha_inicio, fecha_final)

In [14]:
pais = 'LATAM'
nivel_4 = 'id_city'
#id_city = 'id_city'
name_city = 'name_city'
country_city = 'country_city'
filtro_variable = "n_obs"
filtro_num = "10"

In [15]:
#Computar serie de usuarios homogenea
filtro_variable2 = 'n_obs_madrugada'
filtro_variable22 = 'n_obs_noche'
filtro_num2 = 4
#consulta athena
tabla_usuarios_distancia =  f'historico_usuarios_cities_{pais}'   
query_filtro = (f''' SELECT caid,
                            count(*) as count_filtro
                     FROM {tabla_usuarios_distancia} 
                     WHERE (({tabla_usuarios_distancia}.{filtro_variable2} + {tabla_usuarios_distancia}.{filtro_variable22})>= {filtro_num2})
                     GROUP BY caid
                     HAVING count(*)>=30''')
df_filtro = cursor.execute(query_filtro).as_pandas()

In [16]:
start_time = time.time()
for fecha in fechas:
    time_fecha = time.time()
    dia = str(fecha.day)
    mes = str(fecha.month)
    query_fecha = ""
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    tabla_usuarios_distancia =  f'historico_usuarios_cities_{pais}'   
    query_filtro = (f''' SELECT *
                         FROM {tabla_usuarios_distancia} 
                         WHERE ({tabla_usuarios_distancia}.month={mes}
                         AND {tabla_usuarios_distancia}.day={dia} 
                         AND {tabla_usuarios_distancia}.{filtro_variable} >= {filtro_num})''')
    df = cursor.execute(query_filtro).as_pandas()
    df['MasDe1km'] = df['distancia_recorrida'].apply(lambda x: 1 if x>=1 else 0)
    
    
    ##Filtro mas tenue
    
    #Cities  
    df_by_day_4=df.groupby(['month','day', nivel_4, name_city, country_city]).agg({'MasDe1km':'sum','caid':'count'}).reset_index()
    df_by_day_4=df_by_day_4.rename(columns={'caid':'n_users','month':'Mes','day':'Dia'})
    df_by_day_4['%MasDe1km'] = df_by_day_4['MasDe1km']/df_by_day_4['n_users']*100  
    df_median=df[df['MasDe1km']==1].groupby(['month','day',nivel_4]).agg({'distancia_recorrida':'median'}).reset_index()
    df_median=df_median.rename(columns={'distancia_recorrida':'Mediana1KM','month':'Mes','day':'Dia'})
    df_by_day_4=df_by_day_4.merge(df_median, how='left',on=['Mes','Dia',nivel_4])    
    df_by_day_4[nivel_4]=df_by_day_4[nivel_4].astype('int')
    df_by_day_4['Mes']=df_by_day_4['Mes'].astype('int')
    df_by_day_4['Dia']=df_by_day_4['Dia'].astype('int')
    df_by_day_4["Dia"]=df_by_day_4["Dia"].map("{:02}".format)
    df_by_day_4["key4"]=df_by_day_4['Mes'].astype(str)+ df_by_day_4["Dia"].astype(str)
    df_by_day_4['key4']=df_by_day_4['key4'].astype('int')
    df_by_day_4['Dia']=df_by_day_4['Dia'].astype('int')
        
    table_uc = f'historico_cities_{pais}_{codigo_fecha}'
    wr.s3.to_parquet(  # Storing the data and metadata to Data Lake
    df = df_by_day_4,
    dataset = True,
    table = table_uc,
    database = "graphdata",
    path = f"s3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/Results_cities/Temp_{pais}/{table_uc}"
    )
    
    
    ##Filtro severo

    #Cities
    df2=df.merge(df_filtro,how='inner',on='caid')
    df_by_day_44=df2.groupby(['month','day',nivel_4, name_city, country_city]).agg({'MasDe1km':'sum','caid':'count'}).reset_index()
    df_by_day_44=df_by_day_44.rename(columns={'caid':'n_users','month':'Mes','day':'Dia'})
    df_by_day_44['%MasDe1km'] = df_by_day_44['MasDe1km']/df_by_day_44['n_users']*100  
    df_median=df2[df2['MasDe1km']==1].groupby(['month','day',nivel_4]).agg({'distancia_recorrida':'median'}).reset_index()
    df_median=df_median.rename(columns={'distancia_recorrida':'Mediana1KM','month':'Mes','day':'Dia'})
    df_by_day_44=df_by_day_44.merge(df_median, how='left',on=['Mes','Dia',nivel_4])      
    df_by_day_44[nivel_4]=df_by_day_44[nivel_4].astype('int')
    df_by_day_44['Mes']=df_by_day_44['Mes'].astype('int')
    df_by_day_44['Dia']=df_by_day_44['Dia'].astype('int')
    df_by_day_44["Dia"]=df_by_day_44["Dia"].map("{:02}".format)
    df_by_day_44["key4"]=df_by_day_44['Mes'].astype(str)+ df_by_day_44["Dia"].astype(str)
    df_by_day_44['key4']=df_by_day_44['key4'].astype('int')
    df_by_day_44['Dia']=df_by_day_44['Dia'].astype('int')
        
    table_uc = f'historico_cities_{pais}_{codigo_fecha}_fs'
    wr.s3.to_parquet(  # Storing the data and metadata to Data Lake
    df = df_by_day_44,
    dataset = True,
    table = table_uc,
    database = "graphdata",
    path = f"s3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/Results_cities/Temp_{pais}/{table_uc}"
    )  
    

    tiempo = (time.time()-time_fecha)
    print(f"{mes}{dia}, {int(tiempo)} seg")

91, 71 seg
92, 64 seg
93, 63 seg
94, 65 seg
95, 61 seg
96, 57 seg
97, 56 seg
98, 59 seg
99, 61 seg
910, 57 seg
911, 56 seg
912, 52 seg
913, 38 seg
914, 43 seg
915, 36 seg
916, 31 seg
917, 29 seg
918, 25 seg
919, 25 seg
920, 28 seg
921, 50 seg
922, 61 seg
923, 58 seg
924, 62 seg
925, 58 seg
926, 57 seg
927, 58 seg
928, 58 seg
929, 61 seg
930, 49 seg


### Eliminacion de tablas

pais = 'LATAM'
try:
    fecha_inicio = '02-01-2020'
    fecha_final = '02-01-2020'
    fechas = pd.date_range(fecha_inicio, fecha_final)
    for fecha in fechas:
        day = str(fecha.day)
        if len(day) == 1:
            day = f'0{day}'
        codigo_fecha = str(fecha.month) + day
        print(codigo_fecha)
        
        tabla_datos_pais = f"historico_cities_{pais}_{codigo_fecha}"
        query = f"DROP TABLE {tabla_datos_pais}"
        cursor.execute(query)
        
        tabla_datos_pais = f"historico_cities_{pais}_{codigo_fecha}_fs"
        query = f"DROP TABLE {tabla_datos_pais}"
        cursor.execute(query)
        
except:
    print(f" Eliminado ")